In [1]:
%conda activate ./env


Note: you may need to restart the kernel to use updated packages.


In [28]:
%pip install tensorflow

^C
Note: you may need to restart the kernel to use updated packages.


### **Use this Notebook on Google Colab Free GPU**
Go to Google Drive, upload this notebook and dataset folders. Rigth click notebook, choose **Open with -> Google Colab**
<br>
To activate GPU in Colab, go to **Runtime -> Change Runtime Type**. Under **Hardware accelerator** choose **GPU**

### Connect to Google Drive to access Dataset

In [6]:
'''
from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/My Drive/COVID-19 Detection using CT Scan Images'
'''

"\nfrom google.colab import drive\ndrive.mount('/content/drive')\n%cd '/content/drive/My Drive/COVID-19 Detection using CT Scan Images'\n"

In [7]:
%ls

 Volume in drive D is DATA
 Volume Serial Number is DA8A-7D39

 Directory of D:\00_infrrd_documents\projects\COVID-19-Detection-Flask-App-based-on-Chest-X-rays-and-CT-Scans\jupyter notebooks

13-09-2022  13:26    <DIR>          .
13-09-2022  12:09    <DIR>          ..
13-09-2022  12:55                 5 .gitignore
13-09-2022  12:44    <DIR>          .ipynb_checkpoints
13-09-2022  12:52    <DIR>          env
13-09-2022  12:09         2,376,455 InceptionV3_Chest.ipynb
13-09-2022  13:26            23,382 InceptionV3_CT.ipynb
13-09-2022  12:09         2,408,999 RESNET Chest.ipynb
13-09-2022  12:09         2,834,886 RESNET_CT.ipynb
13-09-2022  12:09         2,356,677 VGG chest.ipynb
13-09-2022  12:09         2,898,392 VGG_CT.ipynb
13-09-2022  12:09         2,390,333 Xception_chest.ipynb
13-09-2022  12:09         2,980,996 Xception_CT.ipynb
               9 File(s)     18,270,125 bytes
               4 Dir(s)  571,430,002,688 bytes free


### Import all dependencies

In [8]:
from builtins import range, input

from tensorflow.keras.layers import Input, Lambda, Dense, Flatten, AveragePooling2D, Dropout
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.metrics import confusion_matrix, roc_curve
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt


import cv2
from glob import glob

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.utils import to_categorical

ModuleNotFoundError: No module named 'tensorflow'

### Define Few Parameters

In [9]:
#define size to which images are to be resized
IMAGE_SIZE = [224, 224] # feel free to change depending on dataset

# training config:
epochs = 500
batch_size = 32

#define paths
covid_path = 'data/ct/CT_COVID'
noncovid_path = 'data/ct/CT_NonCOVID'

# Use glob to grab images from path .jpg or jpeg
covid_files = glob(covid_path + '/*')
noncovid_files = glob(noncovid_path + '/*')

NameError: name 'glob' is not defined

In [10]:
# Visualize file variable contents
print("First 5 Covid Files: ",covid_files[0:5])
print("Total Count: ",len(covid_files))
print("First 5 NonCovid Files: ",noncovid_files[0:5])
print("Total Count: ",len(noncovid_files))

NameError: name 'covid_files' is not defined

### Fetch Images and Class Labels from Files (This might take a while)

In [11]:
# Fetch Images and Class Labels from Files
covid_labels = []
noncovid_labels = []

covid_images=[]
noncovid_images=[]

for i in range(len(covid_files)):
  image = cv2.imread(covid_files[i]) # read file 
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # arrange format as per kera
  image = cv2.resize(image,(224,224)) # resize as per model
  covid_images.append(image) # append image
  covid_labels.append('CT_COVID') # append class label
for i in range(len(noncovid_files)):
  image = cv2.imread(noncovid_files[i])
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  image = cv2.resize(image,(224,224))
  noncovid_images.append(image)
  noncovid_labels.append('CT_NonCOVID')

NameError: name 'covid_files' is not defined

### Visualize First 40 Images from Data set

In [12]:
# look at a random image for fun
def plot_images(images, title):
    nrows, ncols = 5, 8
    figsize = [10, 6]

    fig, ax = plt.subplots(nrows=nrows, ncols=ncols, figsize=figsize, facecolor=(1, 1, 1))

    for i, axi in enumerate(ax.flat):
        axi.imshow(images[i])
        axi.set_axis_off()

    plt.suptitle(title, fontsize=24)
    plt.tight_layout(pad=0.2, rect=[0, 0, 1, 0.9])
    plt.show()
plot_images(covid_images, 'Positive COVID-19 CT Scan')
plot_images(noncovid_images, 'Negative COVID-19 CT Scan')

NameError: name 'plt' is not defined

### **Normalization**
#### Model takes images in the form of array of pixels. Hence convert into array and *normalize*

In [13]:
# Convert to array and Normalize to interval of [0,1]
covid_images = np.array(covid_images) / 255
noncovid_images = np.array(noncovid_images) / 255

NameError: name 'np' is not defined

### **Train Test Split**

In [14]:
# split into training and testing
covid_x_train, covid_x_test, covid_y_train, covid_y_test = train_test_split(
    covid_images, covid_labels, test_size=0.2)
noncovid_x_train, noncovid_x_test, noncovid_y_train, noncovid_y_test = train_test_split(
    noncovid_images, noncovid_labels, test_size=0.2)


X_train = np.concatenate((noncovid_x_train, covid_x_train), axis=0)
X_test = np.concatenate((noncovid_x_test, covid_x_test), axis=0)
y_train = np.concatenate((noncovid_y_train, covid_y_train), axis=0)
y_test = np.concatenate((noncovid_y_test, covid_y_test), axis=0)

# make labels into categories - either 0 or 1
y_train = LabelBinarizer().fit_transform(y_train)
y_train = to_categorical(y_train)

y_test = LabelBinarizer().fit_transform(y_test)
y_test = to_categorical(y_test)

NameError: name 'train_test_split' is not defined

### Visualize a few images from Training and Test sets

In [15]:
plot_images(covid_x_train, 'X_train')
plot_images(covid_x_test, 'X_test')
# y_train and y_test contain class lables 0 and 1 representing COVID and NonCOVID for X_train and X_test

NameError: name 'covid_x_train' is not defined

### **Building and Visualizing model**

In [16]:
inception = InceptionV3(weights="imagenet", include_top=False,
    input_tensor=Input(shape=(224, 224, 3)))

outputs = inception.output
outputs = Flatten(name="flatten")(outputs)
outputs = Dropout(0.5)(outputs)
outputs = Dense(2, activation="softmax")(outputs)

model = Model(inputs=inception.input, outputs=outputs)

for layer in inception.layers:
    layer.trainable = False

model.compile(
        loss='categorical_crossentropy', 
        optimizer='adam', 
        metrics=['accuracy']
)

NameError: name 'InceptionV3' is not defined

In [17]:
# Visualize Model
model.summary()

NameError: name 'model' is not defined

### **Image Augmentation**
#### To train on images at different positions, angles, flips, etc

In [18]:
train_aug = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

NameError: name 'ImageDataGenerator' is not defined

### **Training the model**

In [19]:
history = model.fit(train_aug.flow(X_train, y_train, batch_size=batch_size),
                    validation_data=(X_test, y_test),
                    validation_steps=len(X_test) / batch_size,
                    steps_per_epoch=len(X_train) / batch_size,
                    epochs=epochs)

NameError: name 'model' is not defined

In [20]:
# Save Model and Weights
model.save('inception_ct.h5')
model.save_weights('inception_weights_ct.hdf5')

NameError: name 'model' is not defined

In [21]:
# Load saved model
model = load_model('inception_ct.h5')

NameError: name 'load_model' is not defined

### **Making Predicions**

In [22]:
y_pred = model.predict(X_test, batch_size=batch_size)

NameError: name 'model' is not defined

### Visulaizing First 10 predictions

In [23]:
prediction=y_pred[0:10]
for index, probability in enumerate(prediction):
  if probability[1] > 0.5:
        plt.title('%.2f' % (probability[1]*100) + '% COVID')
  else:
        plt.title('%.2f' % ((1-probability[1])*100) + '% NonCOVID')
  plt.style.reload_library
  plt.imshow(X_test[index])
  plt.show()

NameError: name 'y_pred' is not defined

In [24]:
# Convert to Binary classes
y_pred_bin = np.argmax(y_pred, axis=1)
y_test_bin = np.argmax(y_test, axis=1)

NameError: name 'np' is not defined

### Plot ROC Curve

In [25]:
fpr, tpr, thresholds = roc_curve(y_test_bin, y_pred_bin)
plt.plot(fpr, tpr)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.rcParams['font.size'] = 12
plt.title('ROC curve for our model')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.grid(True)

NameError: name 'roc_curve' is not defined

### Plot Confusion Matrix

In [26]:
def plot_confusion_matrix(normalize):
  classes = ['COVID','NonCOVID']
  tick_marks = [0.5,1.5]
  cn = confusion_matrix(y_test_bin, y_pred_bin,normalize=normalize)
  sns.heatmap(cn,cmap='plasma',annot=True)
  plt.xticks(tick_marks, classes)
  plt.yticks(tick_marks, classes)
  plt.title('Confusion Matrix')
  plt.ylabel('True label')
  plt.xlabel('Predicted label')
  plt.show()

print('Confusion Matrix without Normalization')
plot_confusion_matrix(normalize=None)

print('Confusion Matrix with Normalized Values')
plot_confusion_matrix(normalize='true')

Confusion Matrix without Normalization


NameError: name 'confusion_matrix' is not defined

### **Classification Report**

In [27]:
from sklearn.metrics import classification_report
print(classification_report(y_test_bin, y_pred_bin))

NameError: name 'y_test_bin' is not defined

### **Accuracy and Loss Plots**

In [ ]:
plt.figure(figsize=(10,10))

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])

plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')

plt.legend(['Training', 'Testing'])
plt.savefig('inception_ct_accuracy.png')
plt.show()


In [ ]:
plt.figure(figsize=(10,10))

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])

plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')

plt.legend(['Training', 'Testing'])
plt.savefig('inception_ct_loss.png')
plt.show()